In [1]:
import numpy as np
import pandas as pd

import logistic_regression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

# Aids

In [2]:
aids = pd.read_csv('data/aids.csv')

y = aids['target']
X = aids.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [3]:
X_train.shape[1]

20

In [6]:
lr = logistic_regression.LogisticRegressor()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
lr.confusion_matrix(X_test, y_test)

array([[ 34,  16],
       [ 64, 314]], dtype=int64)

In [7]:
X_test.columns

Index(['time', 'trt', 'age', 'wtkg', 'hemo', 'homo', 'drugs', 'karnof',
       'oprior', 'z30', 'preanti', 'race', 'gender', 'symptom', 'treat',
       'offtrt', 'cd40', 'cd420', 'cd80', 'cd820'],
      dtype='object')

In [8]:
sklearn_lr = LogisticRegression(max_iter=10000, tol=1e-3, solver='lbfgs')
sklearn_lr.fit(X, y)
sklearn_lr.score(X, y)
y_pred = sklearn_lr.predict(X)

In [9]:
print(f"Our implementation\n{lr.confusion_matrix(X, y)}")
print(f"Sklearn implementation\n{confusion_matrix(y, y_pred)}")

Our implementation
[[ 182   73]
 [ 339 1545]]
Sklearn implementation
[[1531   87]
 [ 190  331]]


In [12]:
print(f"Accuracy of our implementation: {lr.accuracy(X_test, y_test)}")
print(f"Accuracy of sklearn implementation: {sklearn_lr.score(X_test, y_test)}")

Accuracy of our implementation: 0.8130841121495327
Accuracy of sklearn implementation: 0.8808411214953271
